<a href="https://colab.research.google.com/github/SankirthGunnam/pet-breed-classifier/blob/main/cracked_wall_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 📦 INSTALL DEPENDENCIES
!pip install -q fastai kaggle gradio
from google.colab import files

# 📥 UPLOAD KAGGLE API KEY
uploaded = files.upload()  # Upload `kaggle.json`
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.0 MB/s eta 0:00:00


Saving kaggle.json to kaggle.json


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnavr10880/concrete-crack-images-for-classification")
print("Path to dataset files:", path)
!ls -l {path}

Path to dataset files: /kaggle/input/concrete-crack-images-for-classification
total 0
drwxr-sr-x 2 1000 1000 0 Jun  2 16:39 Negative
drwxr-sr-x 2 1000 1000 0 Jun  2 16:40 Positive


In [3]:
# 📁 IMPORT LIBRARIES
from fastai.vision.all import *
import zipfile, os
import shutil # Import shutil for cross-device moves

# 📂 UNZIP DATASET
# The dataset is already extracted by kagglehub to the path printed above.
# We will now move the contents to the sample_data folder.

# 🧹 PREPARE DATA FOLDERS
# Define the source and destination paths
source_path = path
# Create the destination directory if it doesn't exist

# Remove existing 'cracked' and 'normal' directories if they exist
cracked_path = '/kaggle/input/concrete-crack-images-for-classification/Positive'
normal_path = '/kaggle/input/concrete-crack-images-for-classification/Negative'


# 📊 LOAD DATA
path = Path('/kaggle/input/concrete-crack-images-for-classification')
dls = ImageDataLoaders.from_folder(
    path, train='.', valid_pct=0.2, seed=42,
    item_tfms=Resize(224),
    batch_tfms=aug_transforms()
)

# 🧠 TRAIN MODEL
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)

# 💾 EXPORT MODEL
learn.export('/tmp/concrete_crack_classifier.pkl')

# 📤 DOWNLOAD .PKL
files.download('/tmp/concrete_crack_classifier.pkl')

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 60.7MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.035105,0.012016,0.997125,03:35


epoch,train_loss,valid_loss,accuracy,time
0,0.014587,0.004136,0.998750,03:17
1,0.010831,0.003990,0.999375,03:18
2,0.006575,0.004805,0.999000,03:17
3,0.003679,0.003337,0.999375,03:17


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# 📁 IMPORT LIBRARIES
import gradio as gr

# 📥 UPLOAD THE EXPORTED MODEL FILE
from google.colab import files
uploaded = files.upload()  # Upload `concrete_crack_classifier.pkl`

# 📦 LOAD MODEL
learn = load_learner('/tmp/concrete_crack_classifier.pkl')

# 🔍 PREDICTION FUNCTION
def classify_concrete(img):
    pred, idx, probs = learn.predict(img)
    return {learn.dls.vocab[0]: float(probs[0]), learn.dls.vocab[1]: float(probs[1])}

# 🖼️ GRADIO INTERFACE
gr.Interface(
    fn=classify_concrete,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Concrete Crack Detector",
    description="Upload a concrete surface image to detect cracks"
).launch()


Saving crackedWall.jpeg to crackedWall.jpeg


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3a53c5742a9d06472.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
